In [10]:
import pandas as pd #pandas library for dataframe manipulation
import numpy as np #algebra
import matplotlib.pyplot as plt #plotting graphs
%matplotlib inline
import seaborn as sns


aisles = pd.read_csv('C:\\Users\\kj.park\\Desktop\Class\\Instacart\\aisles.csv')
order_products_prior = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\order_products__prior.csv")
orders = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\orders.csv")
products = pd.read_csv("C:\\Users\\kj.park\Desktop\\Class\\Instacart\\products.csv")
order_products_train = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\order_products__train.csv")
departments = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\departments.csv")

In [11]:
train_set = orders[orders['eval_set']=='train']
train_set = pd.merge(train_set, order_products_train, on='order_id', how='inner')
train_set = pd.merge(train_set, products, on = 'product_id', how = 'inner')
train_set = pd.merge(train_set, departments, on = 'department_id', how = 'inner')
train_set = pd.merge(train_set, aisles, on = 'aisle_id', how = 'inner')
print(train_set.shape)
train_set = train_set.sample(frac=.05)
print(train_set.shape)

(1384617, 15)
(69231, 15)


In [12]:
#This will serve  as the "train" for the model
prior_set = orders[orders['eval_set']=='prior']
prior_set = prior_set.merge(order_products_prior, on='order_id')
prior_set = pd.merge(prior_set, products, on = 'product_id', how = 'inner')
prior_set = pd.merge(prior_set, departments, on = 'department_id', how = 'inner')
prior_set = pd.merge(prior_set, aisles, on = 'aisle_id', how = 'inner')
print(prior_set.shape)
prior_set = prior_set.sample(frac=.001)
print(prior_set.shape)

(32434489, 15)
(32434, 15)


In [13]:
#IGnore
non_text = [',','-',' ','.','&','%','#', '(',')','[',']','*']
for symbols in non_text:
    train_set['product_name'] = train_set['product_name'].str.replace(symbols, '')
    prior_set['product_name'] = prior_set['product_name'].str.replace(symbols, '')
train_set.info()
prior_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69231 entries, 694415 to 924719
Data columns (total 15 columns):
order_id                  69231 non-null int64
user_id                   69231 non-null int64
eval_set                  69231 non-null object
order_number              69231 non-null int64
order_dow                 69231 non-null int64
order_hour_of_day         69231 non-null int64
days_since_prior_order    69231 non-null float64
product_id                69231 non-null int64
add_to_cart_order         69231 non-null int64
reordered                 69231 non-null int64
product_name              69231 non-null object
aisle_id                  69231 non-null int64
department_id             69231 non-null int64
department                69231 non-null object
aisle                     69231 non-null object
dtypes: float64(1), int64(10), object(4)
memory usage: 8.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434 entries, 18337041 to 956300
Data columns (total 15 colu

In [14]:
#Train Set Preparation: using train_set to prepare and check against the "prior set"
train = train_set.loc[:,('user_id','order_id','order_number', 'product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
train = pd.get_dummies(train)
train['days_since_prior_order'] = train['days_since_prior_order'].fillna(value=0)


In [15]:
#Train Set Preparation: using train_set to prepare and check against the "prior set"
test = prior_set.loc[:,('user_id','order_id','order_number', 'product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
test = pd.get_dummies(test)
test['days_since_prior_order'] = test['days_since_prior_order'].fillna(value=0)



In [16]:
train.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,product_name_0CalorieStrawberryDragonfruitWaterBeverage,product_name_0FatBlackCherryGreekYogurty,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
694415,37256,1622599,34,3,13,7.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
97771,160246,1896758,14,1,12,7.0,11,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1199048,202781,2111036,25,2,11,30.0,34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483666,169819,1999653,31,5,10,5.0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,0
406745,53385,3109527,11,0,13,29.0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train = train[test]
train = train[test.columns]
train.head

ValueError: Must pass DataFrame with boolean values only

In [ ]:
train_X = train.loc[:, train.columns != 'reordered']
train_y = train.loc[:,('reordered')]
print(train_X.shape)
print(train_y.shape)
test_X = test.loc[:, test.columns != 'reordered']
test_y = test.loc[:,('reordered')]
print(test_X.shape)
print(test_y.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
LogReg = LogisticRegression()

In [ ]:
LogReg = LogisticRegression()

In [ ]:
LogReg.fit(train_X, train_y).score(test_X, test_y)

#.predict(test_X)
#test_y.head()

In [ ]:
#IGnore
non_text = [',','-',' ','?','.','&','%','#', '(',')','[',']','*','^','$','@','=','+','!','®']
for symbols in non_text:
    train_set['product_name'] = train_set['product_name'].str.replace(symbols, '')
    prior_set['product_name'] = prior_set['product_name'].str.replace(symbols, '')
train_set.info()
prior_set.info()